## Deliverable 2. Create a Customer Travel Destinations Map.

In [29]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [21]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Santa Cruz,BO,-17.8000,-63.1667,78.26,78,75,6.91,broken clouds
1,1,Gradignan,FR,44.7726,-0.6139,48.18,90,75,1.01,broken clouds
2,2,Bredasdorp,ZA,-34.5322,20.0403,59.29,94,100,10.00,moderate rain
3,3,Albany,US,42.6001,-73.9662,38.80,59,100,6.13,overcast clouds
4,4,Bluff,NZ,-46.6000,168.3333,55.80,54,13,19.22,few clouds


In [22]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [23]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Santa Cruz,BO,-17.8000,-63.1667,78.26,78,75,6.91,broken clouds
8,8,Senador Pompeu,BR,-5.5881,-39.3717,82.04,65,23,9.13,few clouds
9,9,Majene,ID,-3.5403,118.9707,80.91,82,100,8.39,very heavy rain
10,10,Yurimaguas,PE,-5.9000,-76.0833,80.76,89,40,3.44,heavy intensity rain
11,11,Atuona,PF,-9.8000,-139.0333,78.84,74,8,18.32,clear sky
14,14,Kapaa,US,22.0752,-159.3190,79.12,68,40,5.01,light rain
16,16,Lorengau,PG,-2.0226,147.2712,86.47,72,97,17.34,moderate rain
20,20,Sao Filipe,CV,14.8961,-24.4956,75.29,74,14,10.56,few clouds
23,23,Rikitea,PF,-23.1203,-134.9692,76.78,72,100,6.42,overcast clouds
29,29,Kavieng,PG,-2.5744,150.7967,83.50,75,86,7.25,light rain


In [24]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [25]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# No empty rows
clean_df = preferred_cities_df

In [26]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Santa Cruz,BO,78.26,broken clouds,-17.8000,-63.1667,
8,Senador Pompeu,BR,82.04,few clouds,-5.5881,-39.3717,
9,Majene,ID,80.91,very heavy rain,-3.5403,118.9707,
10,Yurimaguas,PE,80.76,heavy intensity rain,-5.9000,-76.0833,
11,Atuona,PF,78.84,clear sky,-9.8000,-139.0333,
14,Kapaa,US,79.12,light rain,22.0752,-159.3190,
16,Lorengau,PG,86.47,moderate rain,-2.0226,147.2712,
20,Sao Filipe,CV,75.29,few clouds,14.8961,-24.4956,
23,Rikitea,PF,76.78,overcast clouds,-23.1203,-134.9692,
29,Kavieng,PG,83.50,light rain,-2.5744,150.7967,


In [32]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [33]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df=hotel_df.dropna()
clean_hotel_df.count()

City                   170
Country                170
Max Temp               170
Current Description    170
Lat                    170
Lng                    170
Hotel Name             170
dtype: int64

In [34]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Santa Cruz,BO,78.26,broken clouds,-17.8000,-63.1667,Aviador Hotel Boutique
8,Senador Pompeu,BR,82.04,few clouds,-5.5881,-39.3717,Pousada Ki-Delicia
9,Majene,ID,80.91,very heavy rain,-3.5403,118.9707,Kost Putri RABBANI
10,Yurimaguas,PE,80.76,heavy intensity rain,-5.9000,-76.0833,Rio Huallaga Hotel & Business Center
11,Atuona,PF,78.84,clear sky,-9.8000,-139.0333,Villa Enata


In [35]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [36]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Current Weather</b></dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [37]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))